<h1 style="text-align: center">
<div style="color: #DD3403; font-size: 60%">Data Science DISCOVERY MicroProject</div>
<span style="">MicroProject: Image Steganography with DataFrames</span>
<div style="font-size: 60%;"><a href="https://discovery.cs.illinois.edu/microproject/image-steganography-with-dataframes/">https://discovery.cs.illinois.edu/microproject/image-steganography-with-dataframes/</a></div>
</h1>

<hr style="color: #DD3403;">

## Overview

**Steganography** describes the technique of hiding data within secondary, usually ordinary, data to avoid detection.  For example, an ordinary PNG image might look like a picture to us -- but, hidden inside of the image, is a special encoding that reveals hidden data that otherwise goes undetected.

In this MicroProject, you will explore steganography by decoding a message secretly hidden in an image just for you.  Let's nerd out!

<hr style="color: #DD3403;">

## Part 0: Using the DISCOVERY Library

Identical to the Mosaic Project, we have provided you the DISCOVERY library that loads an image from disk and converts the image into a DataFrame containing the `x`, `y`, `r`, `g`, and `b` values for every pixel.

The "Block I" image is included in this MicroProject and is just an ordinary Illini "Block I" (open it up and check it out).  The following code imports the DISCOVERY library and loads the "Block I" image called `i.png`:

In [0]:
import DISCOVERY

df = DISCOVERY.df_image("i.png")
df

In [0]:
### TEST CASE for Part 0: Using the DISCOVERY Library
tada = "\N{PARTY POPPER}"

assert("df" in vars())
assert(len(df) == 73902)
print(f"{tada} All Tests Passed! {tada}")

<hr style="color: #DD3403;">

## Part 1: Steganography

In the Journal of Vision Research (June 2000), Andrew Stockman and Lindsay T. Sharpe's *["The spectral sensitivities of the middle- and long-wavelength-sensitive cones derived from measurements in observers of known genotype"](https://www.sciencedirect.com/science/article/pii/S0042698900000213)* presents an incredibly detailed study of the sensitivity that the three types of cones have to various wavelengths of light.

Their work quantifies that the human eye is far more sensitive to green and red light than to blue light.  Given this finding, we may be able to slightly change the blue component of an image without it being detected, so we will limit our modifications to be only to the blue value, and only making very small adjustments.

To make sure we never need to change the `b` value significantly, we'll hide our message by checking if the `b` value is "even" or "odd".  Based on this knowledge, our steganography algorithm for this MicroProject will be:

1. We will consider **all columns** but **only the first 27 rows** of any image (`y=0` to `y=26`).
2. **Exactly one** `b` value in the first 27 rows of each column will be an **even value**.   All other `b` values in the first 27 rows will be an odd value.
3. The row in which the even value was found will represent the letter in our secret, hidden message.
    - The even `b` value in row `y=0` represents an `A`,
    - The even `b` value in row `y=1` represents a `B`,
    - The even `b` value in row `y=2` represents a `C`,
    - ...and so on, until...
    - The even `b` value in row `y=25` represents a `Z`, and
    - The even `b` value in row `y=26` represents a space (` `).

*(This is only one possible steganography algorithm among many, but it should succeed in being an unnoticeable change to an image -- and it should be simple to process!)*

### Finding the First 27 Rows

Using the image `df` loaded earlier, first filter the DataFrame to **only the first 27 rows** (`y=0` to `y=26`) making sure to keep all the columns:

In [0]:
# Filter the DataFrame to only the first 26 rows:
...

In [0]:
### TEST CASE for Part 1: Finding the First 27 Rows
tada = "\N{PARTY POPPER}"
assert(max(df.y) == 26), "The maximum value for `y` should be 26 (y=0...26) to consider only the first 27 rows."
assert(len(df) == 6102)

# ...and copy the DataFrame to avoid warnings about the slice:
df = df.copy()

print(f"{tada} All Tests Passed! {tada}")

### Label the Pixels with an Even `b` Value

Now that you have a DataFrame with only the rows we're interested in, we need to find the one row in each column that has an **even value**.  To detect if a value is even, we can use the "modulo" operation.  The "modulo" operation -- denoted by the percentage operator ($\%$) -- is the integer remainder after doing integer/long division.

For example:
- $7 / 4$ has us divide $7$ into $4$.  The result of integer/long division is "$1$ remainder $3$".  Therefore, $7 \% 4 = 3$ (the remainder).
- Similarly, $43 / 11$ is "$3$ remainder $10$".  Therefore, $43 \% 11 = 10$ (since 10 is the remainder).
- Finally, $11 / 2$ is "$5$ remainder $1$".   Therefore, $11 \% 2 = 1$  (since 1 is the remainder).

When we "modulo" by 2, there's a very special property to determine if we have an even or odd number:
- Even numbers ALWAYS have a remainder of `0`
- Odd numbers ALWAYS have a remainder of `1`

Create a new column called `mod2` that stores the result of the operation `df.b % 2` to find if the blue value of each pixel is even or odd:

In [0]:
# Create a new column `mod2`:
...

In [0]:
### TEST CASE for Part 1: Label the Pixels with an Even b Value
tada = "\N{PARTY POPPER}"
assert("mod2" in df), "You must have the column `mod2` in your DataFrame"
assert(sum(df.mod2) == 6074), "You should have 6,074 odd blue pixels among your 6,102 pixels.  Check your mod2 logic."
print(f"{tada} All Tests Passed! {tada}")

### Filter to Include ONLY Even-Valued Blue Pixels

Our steganography algorithm hides our message in the cells with **even values for the blue pixels**.  Using your `mod2` column you just created and your knowledge of the result of the `mod2` operation, filter your DataFrame to only contain the pixels with an even value in the blue pixel:

In [0]:
# Filter your DataFrame to contain ONLY even-valued blue pixels:
...

In [0]:
### TEST CASE for Part 1: Filter your DataFrame to contain ONLY even-valued blue pixels
tada = "\N{PARTY POPPER}"
assert("mod2" in df), "You must have the column `mod2` in your DataFrame"
assert(sum(df.mod2) == 0), "You should have no odd-valued blue pixels"
assert(len(df) == 28), "You should have only 28 pixels"

# ...and copy the DataFrame to avoid warnings about the slice:
df = df.copy()

print(f"{tada} All Tests Passed! {tada}")

<hr style="color: #DD3403;">

## Part 2: Uncovering the Hidden Message

You now have a DataFrame where you have found **exactly one row for every column** and that row tells us the letter in our hidden message.  Let's now decode the message!

### Creating a Character from a Number

In Python, the `chr` function will translate a UTF-8 character code into a letter.  The UTF-8 is an international standard and defines the translation from binary data into letters.  For example, UTF-8 defines that the letter `"A"` is encoded by the number `65`, `"B"` is encoded by the number `66`, and so on.

We can see this in action by running the following cells:

In [0]:
chr(65)

In [0]:
chr(87)

Let's use the fact this is already built into Python (and all other programming languages) to do the work for us!  To set up for using the `chr` function, add a new column to your DataFrame called `charCode`.

The `charCode` column can simply add `65` to the `y` value of each row so that `y=0` has a `charCode` of `65`, `y=1` has a `charCode` of `66`, and so on.  Create this new column below:

In [0]:
# Add the column "charCode":
...

In [0]:
### TEST CASE for Part 2: Creating a Character from a Number
tada = "\N{PARTY POPPER}"
assert("charCode" in df), "You must have the column `charCode` in your DataFrame"
assert(min(df.charCode) == 65), "The smallest charCode value must be 65 (you have a smaller value in your DataFrame)"
assert(max(df.charCode) == 91), "The largest charCode value must be 91 (you have a larger value in your DataFrame)"
print(f"{tada} All Tests Passed! {tada}")

### Apply the `chr` Function to Every Row

Occasionally, we will need to interface between standard Python and DataFrames.  The `chr` function is an Python function, and not a DataFrame operation, so we will need to "apply" this function to our DataFrame.

To do this, we can use the DataFrame `apply` operation to apply a function to every row.  To use the `chr` function on every value in the `charCode` column, the following syntax will do exactly that:

> ```py
> # Run the `chr` function for each row, using the value stored in the column `charCode`:
> df["charCode"].apply(chr)
> ```

Store the result of the code above in a new column called `character`:

In [0]:
# Create a new column `character` by using the apply function:
df["character"] = ...
df

In [0]:
### TEST CASE for Part 2: Applying the chr Function
tada = "\N{PARTY POPPER}"
assert("character" in df), "You must have the column `character` in your DataFrame"
assert(len(df[df.character == "A"] == 1)), "Your character translation is incorrect"
assert(len(df[df.character == "E"] == 4)), "Your character translation is incorrect"
assert(len(df[df.character == "S"] == 2)), "Your character translation is incorrect"

print(f"{tada} All Tests Passed! {tada}")

### Concatenate the Entire String

We're almost done!!  If you look at your DataFrame, and read down the `character` column, you will see the hidden message -- but we can do better -- we want to combine everything together into one string!

To combine all string columns together across an entire DataFrame, the `str.cat()` function will **concatenate** all the strings together into one large string.  For example:

> ```py
> message = df.columnName.str.cat()
> message    # combines all `columnName` strings together across the full DataFrame
> ```

Create a variable `message` that combines all of the `character`s together across the full DataFrame:

In [0]:
message = ...
message

### Fix the Spaces


Oh no, you may have saw this coming!  There space character is not correct -- it looks like it was mapped to the wrong character!

Use `message.replace("[", " ")` to replace all `"["` characters with a space `" "` character:

In [0]:
# Fix the spaces in our string:
message = ...
message

In [0]:
### TEST CASE for Part 2: Concatenate the Entire String and Fix the Spaces
tada = "\N{PARTY POPPER}"
assert("message" in vars()), "You must define the variable `message`"
assert(len(message) == 28), "Your message must be 28 characters long"
assert(message[1] + message[5] + message[21] == "OOM"), "Your message appears incorrect"
assert(message[3] == " "), "Your message must have spaces for spaces (not '[')"
print(f"{tada} All Tests Passed! {tada}")


<hr style="color: #DD3403;">

## Part 3: Make a Reuseable Function

Often, programmers will do a task once to understand it and then combine the steps together into a function that can be used with different inputs.

Complete the following function, `decodeHiddenMessage`, that takes in a filename (like `i.png`) and returns the hidden message contained within in.  *(This function should simply combine all the steps in the previous parts into a single function, but with the provided file name `fileName` instead of `i.png`.*)

In [0]:
def decodeHiddenMessage(fileName):
  ...

Let's first make sure your function work with the same input we've worked with already:

In [0]:
decodeHiddenMessage("i.png")

If that looks good, I've hidden a few more messages for you:

In [0]:
decodeHiddenMessage("discovery.png")

You can now use this cell for any other file if you want:

In [0]:
### TEST CASE for Part 3: Reuseable Function

tada = "\N{PARTY POPPER}"
assert("decodeHiddenMessage" in vars()), "You must define the function `decodeHiddenMessage`"
__test_msg = decodeHiddenMessage("i.png")
assert(len(__test_msg) == 28), "Your message must be 28 characters long"
assert(__test_msg[1] + __test_msg[5] + __test_msg[21] == "OOM"), "Your message appears incorrect"

__test_msg = decodeHiddenMessage("discovery.png")
assert(len(__test_msg) == 79), "Your message must be 28 characters long"
assert(__test_msg[1] + __test_msg[5] + __test_msg[21] == "ORC"), "Your message appears incorrect"

print(f"{tada} All Tests Passed! {tada}")

<hr style="color: #DD3403;">

## Submission

You're almost done!  All you need to do is to commit your lab to GitHub and run the GitHub Actions Grader:

1.  ⚠️ **Make certain to save your work.** ⚠️ To do this, go to **File => Save All**

2.  After you have saved, exit this notebook and return to https://discovery.cs.illinois.edu/microproject/image-steganography-with-dataframes/ and complete the section **"Commit and Grade Your Notebook"**.

3. If you see a 100% grade result on your GitHub Action, you've completed this MicroProject! 🎉